In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import fl_utils.fl_interpretable_functions as interpret
import fl_utils.fl_riskslim as slim
import fl_utils.fl_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\fl-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY_cleaned").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL_cleaned").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['misdemeanor_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['misdemeanor_two_year'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003, 0.004, 0.005]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [4]:
np.mean(cart_summary['KY_score']), np.mean(ebm_summary['KY_score'])

(0.5687404614725384, 0.6419825969257767)

### Lasso Stumps

In [4]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY_stumps").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL_stumps").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [5]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.08, 
                                        cols, 
                                        816)
single_stump_model['features']

['age_at_current_charge24',
 'age_at_current_charge30',
 'age_at_current_charge36',
 'age_at_current_charge61',
 'p_arrest4',
 'p_arrest6',
 'p_charges1',
 'p_charges2',
 'p_charges3',
 'p_violence3',
 'p_felony2',
 'p_misdemeanor2',
 'p_misdemeanor3',
 'p_property3',
 'p_felprop_viol1',
 'p_misdeassult1',
 'p_drug1',
 'p_fta_two_year1',
 'p_pending_charge1',
 'six_month1',
 'one_year1',
 'three_year1',
 'current_violence1']

In [6]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [7]:
stumps_summary['best_params'], np.mean(stumps_summary['KY_score']), np.std(stumps_summary['KY_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.08}, {'C': 0.08}, {'C': 0.08}, {'C': 0.08}, {'C': 0.08}],
 0.6106976566514726,
 0.012136985158485445,
 0.046430146984175844)

### RiskSLIM

In [8]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.06,     
                                        cols, 816)
selected_features = single_stump_model['features']
len(selected_features)

18

In [9]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [10]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'misdemeanor_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
12/26/19 @ 11:29 AM | 233 rows in lookup table
12/26/19 @ 11:29 AM | ------------------------------------------------------------
12/26/19 @ 11:29 AM | runnning initialization procedure
12/26/19 @ 11:29 AM | ------------------------------------------------------------
12/26/19 @ 11:29 AM | CPA produced 2 cuts
12/26/19 @ 11:29 AM | running naive rounding on 24 solutions
12/26/19 @ 11:29 AM | best objective value: 0.5537
12/26/19 @ 11:29 AM | rounding produced 5 integer solutions
12/26/19 @ 11:29 AM | best objective value is 0.5778
12/26/19 @ 11:29 AM | running sequential rounding on 24 solutions
12/26/19 @ 11:29 AM | best objective value: 0.5537
12/26/19 @ 11:29 AM | sequential rounding produced 6 integer solutions
12/26/19 @ 11:29 AM | best objective value: 0.5604
12/26/19 @ 11:29 AM | polishing 11 solutions
12/26/19 @ 11:29 AM | best objective value: 0.5604
12/26/19 @ 11:29 AM | polishing produced 5 integer solutions
12/26/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:29 AM | adding 160 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5604.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:30 AM | adding 154 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5771.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:30 AM | adding 164 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5801.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:30 AM | adding 145 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5716.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:51 AM | adding 140 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5783.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:52 AM | adding 172 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5746.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:52 AM | adding 131 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5736.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:52 AM | adding 170 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5787.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/26/19 @ 11:52 AM | adding 176 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5802.
Tried aggregator 1 time.
Reduced MIP has 20 rows, 40 columns, and 75 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [11]:
np.mean(riskslim_summary['FL_validation']), np.mean(riskslim_summary['KY_score'])

(0.5852060314332693, 0.6041615850098221)

In [12]:
#### save results
summary_misdemeanor2_fl_inter_model = {"cart": cart_summary,
                                       "ebm": ebm_summary,
                                       "stumps": stumps_summary,
                                       "riskslim": riskslim_summary}

In [13]:
path = "./results/interpretable/two-year/"
results = [["Misdemeanor",             
            np.str(round(np.mean(cart_summary['KY_score']),3)) + " (" + np.str(round(np.std(cart_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['KY_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['KY_score']),3)) + " (" + np.str(round(np.std(stumps_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['KY_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['KY_score']),3)) + ")"]]
with open(path + 'two-year-fl-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)